# Intelligent Electoral Bond AI chatbot Assistant

In [1]:
!pip install langchain pypdf faiss-cpu openai tiktoken pinecone-client newsapi-python chromadb apify GitPython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [17]:
from langchain.chains import LLMChain
import os
import openai
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory

# from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.callbacks.manager import CallbackManager

from langchain.agents import initialize_agent, Tool

from langchain.chains import LLMMathChain
from langchain.utilities import GoogleSerperAPIWrapper

Upload the pdf file and set file path

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from tqdm import tqdm

# Specify the directory containing the PDF files
pdf_directory = '/content/drive/MyDrive/EB_Project/data'

# Create an empty list to store the documents
all_documents = []

# Iterate over all files in the directory
for filename in tqdm(os.listdir(pdf_directory)):
    # Check if the file is a PDF
    if filename.endswith('.pdf'):
        # Construct the full file path
        file_path = os.path.join(pdf_directory, filename)

        # Load the PDF using PyPDFLoader
        loader = PyPDFLoader(file_path=file_path)
        documents = loader.load()

        # Split the documents into smaller chunks (optional)
        text_splitter = RecursiveCharacterTextSplitter()
        documents = text_splitter.split_documents(documents)

        # Append the documents to the list
        all_documents.extend(documents)

# Now you have a list of all the documents from all the PDF files
print(f"Total documents loaded: {len(all_documents)}")

100%|██████████| 35/35 [00:09<00:00,  3.60it/s]

Total documents loaded: 215


In [ ]:
all_documents

[Document(page_content='9lf 5939\' Head Office\nShiromani Akali\nNo -SAD/2023/186Dated 10-11-2023\nTo\n8h Binod Kumar,\nSecretary, Election Expenditure Division,\nRoom No -321A, liird Floor,\nElection Commission of India\nNirvachan Sadan Ashoka Road, Ne\\w Delhi-110001\nSubject Submission of Details in respect of electoral bond(s)-Hon\'ble Supreme Court Order dated 2""^\nNovember, 2023-Regarding\nRespected Sir,\nKindly refer to your Letter No - 76/Transparency/2017/PPEMS/Vol II Dt -3"^ November 2023 on the subject\ncited above \'Tour office has sought details of donations received through electoral bonds by the political parties till\n30-09-2023\nWe hereby submit the required information of donations received by us through electoral bonds in our\nCurrent Bank Account maintained in "State bank of India" authorized bank Branch under the scheme "Electoral\nBond Scheme-2018" as under\nSr NoParticulars Details\n1Name of Political Party SHIROMANI AKALI DAL\n2Registered Address of Political P

Provide Openai Key

In [10]:
import os
os.environ['OPENAI_API_KEY'] ='xxxx'
os.environ['SERP_API_KEY']='yyyy'

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(show_progress_bar=True)

vector1 = embeddings.embed_query('Whats your name')

len(vector1)

  0%|          | 0/1 [00:00<?, ?it/s]

1536

In [ ]:
len(vector2)

1536

In [6]:
from langchain.vectorstores import FAISS

index = FAISS.from_documents(all_documents, embeddings)

In [ ]:
index.save_local('/content/drive/MyDrive/EB_Project')

In [12]:
index=FAISS.load_local('/content/drive/MyDrive/EB_Project/vector_faiss', embeddings, allow_dangerous_deserialization=True)

In [ ]:
index.similarity_search_with_relevance_scores(
    "Aam Admi PArty"
)

  0%|          | 0/1 [00:00<?, ?it/s]

[(Document(page_content="Aam Aadmi Party\\\n206, Rouse Avenue, Deen Dayal Upadhyay Marg, New Delhi - 110002\nPhone +91 9718500606, 9818500606, Email info@aamaadmiparty org\nCONFEDENTIAL-ELECTORAL BONDSElection Symbo! Broom\nDate November 15*^ 2023\nTo\nSh Binod Kumar\nThe Secretary,\nRoom no 321A,\nElection Commission of India,\nNirvachan Sadan, Ashoka Road,\nNew Delhi-110001\nSUB SUBMISSION OF DETAILS IN RESPECT OF ELECTORAL BONDS- HON'BLE SUPREME COURT\nORDER DATED 2*^° NOVEMBER 2023\nREF LETTER NO 76/TRANSPARENCY/2017/PPEMS/VOL II dated 03'^'' November 2023\nDear Sir,\nThis IS with respect to the afore-mentioned letter In this regard, as required in aforesaid\nletter, we are enclosing details of Electoral Bonds received till 30 09 2023 as available in\nour records in a sealed envelope\nThanking you\nYours faithfully\nFor AAM AADMI PARTY\nNational Treasurer\n(Narain Dass Gupta)\nNational Treasurer\n/\nPhone 011-23219302, 23219303 I Email contact@aamaadmiparty org\nwww aamaadmiparty o

In [21]:
llm = ChatOpenAI()

prompt_template = """Answer the question as News Critic
Context: {context}


{chat_history}
Human: {question}
Assistant:"""

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template=prompt_template,
    # partial_variables={"format_instructions": parser.get_format_instructions()},
)


memory = ConversationBufferWindowMemory(memory_key="chat_history", input_key="question", k=5)
handler = StdOutCallbackHandler()
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

retriever = index.as_retriever(
search_type="similarity",
search_kwargs={'k': 10}
    )

In [10]:
from langchain.utilities import GoogleSerperAPIWrapper

In [ ]:

question = "Give me total amount COMMUNIST PARTY OF INDIA received."
docs = retriever.get_relevant_documents(question)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


  0%|          | 0/1 [00:00<?, ?it/s]

[Document(page_content='COMMUNIST PARTY OF INDIA (MARXIST)\nCentral Committee\nGeneral Secretary Sitaram Yechury\nNovember 5, 2023\nShri Binod Kumar\nSecretary\nElection Expenditure Division,\nRoom No 321 A, Third Floor\nNirvachan Sadan\nNew Delhi 110 001\nDear Sir,\nThis IS with reference to your letter No 76/Transparency/2017/PPEMSA^ol II\nof November 3, 2023 asking political parties to disclose particulars of the\nelectoral bonds received by them\nThe Communist Party of India (Marxist), as you are aware, has right from the\ninception of the electoral bond scheme voiced its opposition to it We had\ndecided against accepting any donation through the medium of electoral bonds\nIn consonance with this principled stand, the Party has not received any\ndonation through electoral bonds\nFurther, the CPI(M) has not designated any bank account for the said purpose\nYou may also be aware that among the three cases challenging the Electoral\nBonds Scheme pendmg before the Supreme Court, one is

In [ ]:
res=llm_chain.run(question=question, context=docs, callbacks=[handler])
res



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question as News Critic
Context: [Document(page_content='COMMUNIST PARTY OF INDIA (MARXIST)\nCentral Committee\nGeneral Secretary Sitaram Yechury\nNovember 5, 2023\nShri Binod Kumar\nSecretary\nElection Expenditure Division,\nRoom No 321 A, Third Floor\nNirvachan Sadan\nNew Delhi 110 001\nDear Sir,\nThis IS with reference to your letter No 76/Transparency/2017/PPEMSA^ol II\nof November 3, 2023 asking political parties to disclose particulars of the\nelectoral bonds received by them\nThe Communist Party of India (Marxist), as you are aware, has right from the\ninception of the electoral bond scheme voiced its opposition to it We had\ndecided against accepting any donation through the medium of electoral bonds\nIn consonance with this principled stand, the Party has not received any\ndonation through electoral bonds\nFurther, the CPI(M) has not designated any bank account for the said purpose\nYou may also be aware th

'As a News Critic, I would say that the Communist Party of India (CPI) has taken a principled stand against accepting donations through electoral bonds and has not received any such donations. This reflects their commitment to transparency and accountability in political funding.'

In [22]:
question = "Give me total amount BJP received."
docs = retriever.get_relevant_documents(question)
res=llm_chain.run(question=question, context=docs, callbacks=[handler])
res

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Answer the question as News Critic
Context: [Document(page_content='Details of Electoral Bone(s) received FY 2018-19\nPARTICULARS Amount of eachNumbers ofAmount Date ofAmount credited to\nBond Bond(s) Credit\nState Bank of India , Corporate 1,00,000 1010,00,00016 07 2018Account Number 37815704340,\nCentre, State Bank Bhavan Madame Satate Bank of India, Bhubaneswar\nCama Road, Nariman Point, Mam Branch, Bhubaneswar\nMumbai-400021\ndo 10,00,000 990,00,00016 07 2018 do\ndo 10,00,000 550,00,00016 07 2018 do\ndo 10,00,000 101,00,00,00016 07 2018 do\ndo 1,00,00,000 1515,00,00,00016 07 2018 do\ndo 1,00,00,000 2020,00,00,00012 10 2018 do\ndo 1,00,00,000 1010,00,00,00012 10 2018 do\ndo 10,00,000 505,00,00,00012 10 2018 do\ndo 10,00,000 202,00,00,00012 10 2018 do\ndo 1,00,00,000 2222,00,00,00012 10 2018 do\ndo 1,00,00,000 55,00,00,00012 10 2018 do\ndo 1,00,00,000 55,00,00,00012 10 2018 do\ndo 1,00,00,000 3131,00,00,00007 01 2019 do\ndo 

'The Bharatiya Janata Party received a total amount of 22,38,50,000 in the financial year 2020-2021, 2,10,00,02,000 in the financial year 2017-2018, and a total of 4,21,27,51,000 as per the available data.'

In [14]:

# from langchain.tools.retriever import create_retriever_tool

# Define constants for model and vector store paths
# MODEL_PATH = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"
openai_api_key = os.environ.get("OPENAI_API_KEY")
serper_api_key=os.environ.get("SERP_API_KEY")

VECTOR_STORE_PATH = "Downloads/dsp/NLP/vector_faiss/vector_faiss"
# EMBEDDING_MODEL_NAME = "avsolatorio/GIST-large-Embedding-v0"
# os.environ['OPENAI_API_KEY'] =os.environ.get("OPEN_API_KEY")
openai.api_key = openai_api_key
os.environ["SERPER_API_KEY"]=serper_api_key



@cl.cache
def instantiate():

    llm = ChatOpenAI()


    embeddings = OpenAIEmbeddings(show_progress_bar=True)

    # Load vector store
    # index=FAISS.load_local('https://drive.google.com/file/d/1-DgmC8rYns-IRmIGZZF1TcaHztX2i7vq/view?usp=sharing', embeddings, allow_dangerous_deserialization=True)
    index=FAISS.load_local('Downloads/dsp/NLP/vector_faiss/vector_faiss', embeddings,allow_dangerous_deserialization=True)
    return llm , index

llm , vector_store= instantiate()

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
# Set up tools

search = GoogleSerperAPIWrapper()



tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]




# Set up agent
agent = initialize_agent(
    tools,
    llm,
    agent="conversational-react-description",
    verbose=True,
    memory=ConversationBufferWindowMemory(memory_key="chat_history", input_key="question", k=5),
    return_intermediate_steps=True,
)


# Set up conversation chain
@cl.on_chat_start
def main():
    cl.user_session.set(
        "message_history",
        [{"role": "system", "content": "You are a News critic."}],
    )


    prompt_template = """Answer the question as News Critic
    Context: {context}


    {chat_history}
    Human: {question}
    Assistant:"""

    # Create prompt from prompt template
    prompt = PromptTemplate(
        input_variables=["context", "question", "chat_history"],
        template=prompt_template,
        # partial_variables={"format_instructions": parser.get_format_instructions()},
    )


    memory = ConversationBufferWindowMemory(memory_key="chat_history", input_key="question", k=5)
    handler = StdOutCallbackHandler()
    llm_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True,
        memory=memory
    )

    cl.user_session.set("llm_chain", llm_chain)

    retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 10}
        )

    cl.user_session.set("retriever", retriever)
    cl.user_session.set("agent", agent)


@cl.on_message
async def main(message: cl.Message):
    llm_chain = cl.user_session.get("llm_chain")
    retriever= cl.user_session.get("retriever")
    agent = cl.user_session.get("agent")

    # Get the user's message
    user_input = message.content
    docs = retriever.get_relevant_documents(user_input)
    msg = cl.Message(content="")
    await msg.send()
    # Generate response using LLMChain
    response = await llm_chain.arun(question=user_input, context=docs, callbacks=[cl.LangchainCallbackHandler(stream_final_answer=True)],agent=agent)
    # response=await cl.make_async(agent.run)(message.content, callbacks=[cl.LangchainCallbackHandler(stream_final_answer=True)])


    for part in (response):
        if token := part or "":
            await msg.stream_token(token)
    # # Send the response back to the user
    # await message.respond(response)
    await msg.update()

# @cl.on_stop
# def on_stop():
#     print("The user wants to stop the task!")